# RNN Based molucule generation

Laurent Cetinsoy

In this hands-on we want to generate molecule formulas for denovo-drug discovery.

For that we need to use Generative models. Generative models are models which goes beyond classification or simple regression : they are able to generate data that look like previously seens dataset.

There exists a lot of models :

- Bayesian models like graphical models
- Recurrent models (for sequence generation like texte)
- Variational auto encoders
- Generative adversarial models
- Flow and diffusion models


In the hands-on we will start by  trainning a character based RNN to generate smile molecules


We want to feed smile representations of molecules to an RNN.
The basic idea is we will train it to predict the next smile token of a molecule given the previous one.

For instance for the following molecule "CC(=O)NC1=CC=C(O)C=C1" will may give to the model

X = "CC(=O)N"
y = C

and ask the RNN to learn to predict y given X

Like a standard language model !


## RNN Language model


A language model is a model which predict the next token of a sequence given the previous ones :

$ P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p})  $


This model can be learned with a Recurrent neural network

$ y = P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p}) = RNN_{\theta} (X_{t-1}, X_{t-2}, ..., X_{t-p})  $


In order to train such model you need a corpus of data.



There are two main ways to do that : Word level model or character level model

For character level models, an interesting resource is : http://karpathy.github.io/2015/05/21/rnn-effectiveness/



Explain briefly what is the difference between word based language model and character based language model

The primary difference lies in the level of granularity at which these models operate: word-based models focus on words or tokens, while character-based models work with individual characters. 

## Loading the data

Dowload the following dataset : https://github.com/joeymach/Leveraging-VAE-to-generate-molecules

Import pandas and load the first 1000 lines

In [1]:
import pandas as pd

df = pd.read_csv('data/250k_smiles.csv', nrows=1000)

Display the first rows of the dataframe

In [2]:
df.head()

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182


## Processing the data

We need to do the following things :

- convert smile tokens to numbers
- build  smile token sequences and corresponding labels pairs

Compute the biggest smile molecule size

In [3]:
from rdkit import Chem

max_len = df['smiles'].map(len).max()
print('Max length of SMILES: ', max_len)

Max length of SMILES:  106



Code a function **unic_characters(string)** which return the unic characters in a string


In [4]:
def unic_characters(string):
    return list(set(string))

Concatenate all smile string of the pandas dataframe and use **unic_characters** to get the unic_characters

In [5]:
all_smiles = ''.join(df['smiles'])
unic_characters_all_smiles = unic_characters(all_smiles)

Code a function **map_char_to_int(unic_chars)** which returns a dictionnary where each char is assigned an int value.
Add a character to specify the end of the molecule (like "\n")


In [6]:
def map_char_to_int(unic_chars):
    char_to_int = {}
    max_i = 0
    for i, char in enumerate(unic_chars):
        char_to_int[char] = i
        max_i = i
    char_to_int['\n'] = max_i + 1
    return char_to_int

Code a function map_int_to_char(unic_chars) which returns the reverse mapping.

If you want you can merge both functions in a class

In [7]:
def map_int_ot_char(unic_chars):
    int_to_char = {}
    max_i = 0
    for i, char in enumerate(unic_chars):
        int_to_char[i] = char
        max_i = i
    int_to_char[max_i + 1] = '\n'
    return int_to_char

For each smile molecule add the ending token to it

In [8]:
df['smiles'] = df['smiles'].map(lambda x: x + '\n')

## Building the dataset

Now we will create the dataset so that it has the good share for our Keras LSTM model

Remember Keras recurrent models expect a 3D array with shapes (n_examples, seq_len, n_features)



What will be n_features in our case ?

n_features will be the number of unique characters in your character vocabulary.

Code a function **build_X_and_y(string, i_char, seq_lenght)** which takes a string, a **seq_length** number and a position.


It should create X by by getting all character between i and i + seq_length
and create y by getting the character following the X sequence
it returns X and y

Test your function on the following string "" with seq_length = 4 and i = [1, 2, 3]

In [20]:
def build_X_and_y(string, i_char, seq_length):
    l = len(string)
    X = []
    i = i_char
    for i in range(i_char, min(i_char + seq_length, l)):
        X.append(string[i])
    if i_char + seq_length >= l:
        y = "\n"
    else:
        y = string[i_char + seq_length]
    return X, y

In [21]:
build_X_and_y("", 1, 4)

([], '\n')

In [22]:
build_X_and_y("", 2, 4)

([], '\n')

In [23]:
build_X_and_y("", 3, 4)

([], '\n')

By using build_X_and_y and map_char_to_int build a list nameed X_train and a list named y_train

In [120]:
unic_char_int = map_char_to_int(unic_characters_all_smiles)

X_train = []
y_train = []
for i in range(len(df)):
    for i_char in range(len(df['smiles'][i]) - 6):
        X, y = build_X_and_y(df['smiles'][i], i_char, 5)
        X_train.append([unic_char_int[char] for char in X])
        y_train.append(unic_char_int[y])

Create numpy arrays from the lists

In [121]:
import numpy as np

X_train = np.array(X_train)
y_train = np.array(y_train)

Reshape the X numpy array (n_examples, seq_lenght, 1)

In [122]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

X_train.shape

(40242, 5, 1)

Normalize X by dividing each values by the total number of unic characters

In [123]:
nb_unic_chars = len(unic_characters_all_smiles) + 1
X_train = X_train / float(nb_unic_chars)

Import Keras and build (at least) a two layered LSTM network with 128 neurone in each.

You can also add Dropoutlayers

Do you think you should use the return_sequences = True ? If yes, when ?


Add a Dense layer on top with with the appropriate activation function and number of neurones


The parameter return_sequeces is set to True when we want to make sequence to sequence prediction. In our case, we want to predict the next character of the sequence, so we don't need to set it to True. Also, We need to use return_sequeces = True when we want to stack multiple LSTM layers.

In [162]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

model = Sequential()

model.add(LSTM(128, input_shape=X_train[0].shape, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(nb_unic_chars, activation='sigmoid'))

Compile the model with the appropriate loss function and the adam optimizer

In [148]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_28 (LSTM)              (None, 5, 128)            66560     
                                                                 
 dropout_26 (Dropout)        (None, 5, 128)            0         
                                                                 
 lstm_29 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_27 (Dropout)        (None, 128)               0         
                                                                 
 dense_12 (Dense)            (None, 34)                4386      
                                                                 
Total params: 202530 (791.13 KB)
Trainable params: 202530 (791.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Train the model on 20 epochs and 10 examples (yeah you read correctly) and check that the model overfits !

In [149]:
model.fit(X_train[:10], y_train[:10], epochs=20)

Epoch 1/20


1/1 [==============================] - 3s 3s/step - loss: 3.5357
Epoch 2/20
1/1 [==============================] - 0s 9ms/step - loss: 3.5219
Epoch 3/20
1/1 [==============================] - 0s 9ms/step - loss: 3.5064
Epoch 4/20
1/1 [==============================] - 0s 9ms/step - loss: 3.4893
Epoch 5/20
1/1 [==============================] - 0s 9ms/step - loss: 3.4719
Epoch 6/20
1/1 [==============================] - 0s 8ms/step - loss: 3.4569
Epoch 7/20
1/1 [==============================] - 0s 9ms/step - loss: 3.4418
Epoch 8/20
1/1 [==============================] - 0s 8ms/step - loss: 3.4077
Epoch 9/20
1/1 [==============================] - 0s 9ms/step - loss: 3.3881
Epoch 10/20
1/1 [==============================] - 0s 9ms/step - loss: 3.3446
Epoch 11/20
1/1 [==============================] - 0s 10ms/step - loss: 3.3092
Epoch 12/20
1/1 [==============================] - 0s 9ms/step - loss: 3.2448
Epoch 13/20
1/1 [==============================] - 0s 8ms/step - loss: 3.2064
Epoch 

If it does not overfit try to fix data prep and model architecture so it does

The model does overfit

Create a function **make_prediction(seed_start)** which takes a starting string sequence and uses it to generate a molecule


In [178]:
def make_prediction(seed_start):
    import random
    length = random.randint(10, 50)
    while len(seed_start) < length:
        X, _ = build_X_and_y(seed_start, len(seed_start) - 5, 5)
        X = np.array([unic_char_int[char] for char in X]).astype(np.float32)
        X = np.reshape(X, (X.shape[0], 1))
        X = X / float(nb_unic_chars)
        pred = model.predict(np.array([X]), verbose=0)
        seed_start += map_int_ot_char(unic_characters_all_smiles)[np.argmax(pred)]
    return seed_start

generate a molecule of your overfitted model

In [ ]:
make_prediction('CC1=')

'CC1=CC=CC=C1'

Make a model checkpoint so that the model is saved after each epoch
if you train on a plateform and it stops you do not lose your training

In [163]:
from keras.callbacks import ModelCheckpoint

filepath="model/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

Now go to your favorite plateform (colab or something else) and train the dataset on the whole data for 10 epochs and batch size 256

it should take a long time so either follow the class or go take a nap

In [164]:
model.fit(X_train, y_train, epochs=10, callbacks=[checkpoint], batch_size=256)

Epoch 1/10


158/158 [==============================] - ETA: 0s - loss: 2.7791
Epoch 1: loss improved from inf to 2.77912, saving model to model/weights-improvement-01-2.7791.hdf5
158/158 [==============================] - 6s 20ms/step - loss: 2.7791
Epoch 2/10
156/158 [============================>.] - ETA: 0s - loss: 2.6450
Epoch 2: loss improved from 2.77912 to 2.64513, saving model to model/weights-improvement-02-2.6451.hdf5
158/158 [==============================] - 3s 21ms/step - loss: 2.6451
Epoch 3/10
157/158 [============================>.] - ETA: 0s - loss: 2.5455
Epoch 3: loss improved from 2.64513 to 2.54549, saving model to model/weights-improvement-03-2.5455.hdf5
158/158 [==============================] - 3s 21ms/step - loss: 2.5455
Epoch 4/10
157/158 [============================>.] - ETA: 0s - loss: 2.4095
Epoch 4: loss improved from 2.54549 to 2.40964, saving model to model/weights-improvement-04-2.4096.hdf5
158/158 [==============================] - 3s 17ms/step - loss: 2.4096
Epo

Generate between 100 and 1000 molecules.

create a list where molecules have between 10 and 50 atoms

In [185]:
pred_vectorized = np.vectorize(lambda x: make_prediction(x[:5]))

smiles = df['smiles'][:250].to_numpy()

generated = pd.DataFrame()

generated['smiles'] = pred_vectorized(smiles)

With rdkit compute the Quantified Estimated Drug likelyness (QED) of each molecule in this subset

In [186]:
#compute QED

from rdkit.Chem import QED

def compute_qed(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        qed = QED.qed(mol)
    except:
        qed = 0
    return qed

generated['qed'] = generated['smiles'].map(lambda x: compute_qed(x))

generated.head()

[12:25:35] SMILES Parse Error: extra close parentheses while parsing: N#Cc1ccc(CC)c1)cccc1CCCCCCCCCCCCCCCC
[12:25:35] SMILES Parse Error: Failed parsing SMILES 'N#Cc1ccc(CC)c1)cccc1CCCCCCCCCCCCCCCC' for input: 'N#Cc1ccc(CC)c1)cccc1CCCCCCCCCCCCCCCC'
[12:25:35] SMILES Parse Error: unclosed ring for input: 'N#CC1CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC'
[12:25:35] SMILES Parse Error: extra close parentheses while parsing: CC[NH])c1ccc(CC)c1)cccc1CCCCCCCCCCCCCCCCCCCCC
[12:25:35] SMILES Parse Error: Failed parsing SMILES 'CC[NH])c1ccc(CC)c1)cccc1CCCCCCCCCCCCCCCCCCCCC' for input: 'CC[NH])c1ccc(CC)c1)cccc1CCCCCCCCCCCCCCCCCCCCC'
[12:25:35] SMILES Parse Error: extra close parentheses while parsing: COc1ccc(CC)c1)cccc1CCCC
[12:25:35] SMILES Parse Error: Failed parsing SMILES 'COc1ccc(CC)c1)cccc1CCCC' for input: 'COc1ccc(CC)c1)cccc1CCCC'
[12:25:35] SMILES Parse Error: extra close parentheses while parsing: CCOc1ccc(CC)c1)cccc1CCCCCCCCCCCCCCCCCCCCC
[12:25:35] SMILES Parse Error: Failed parsing SMILES '

,smiles,qed
0,CC(C)CCCCCCCCCCCCCC,0.312892
1,C[C@@H](C)CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,0.089990
2,N#Cc1ccc(CC)c1)cccc1CCCCCCCCCCCCCCCC,0.000000
3,CCOC(=O)NCCCCCCCCCCCCCCCCCCCCCCCCCCCC,0.112793
4,N#CC1CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,0.000000


Bonus 1 : Using rdkit, compute the quantitative estimation of drug-likeness (QED) of your generated molecules.

Bonus 2 : try to adapt a transformer model training from hugging face to see if it is better